In [18]:
import numpy as np
import pandas as pd


df = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/BTC_combined_2504_08v4.csv')

df['bar_end'] = pd.to_datetime(df['bar_end'])

df = df.set_index('bar_end')

df = df.drop(columns=[c for c in df.columns if c.lower().startswith('unnamed')], errors='ignore')

df['funding_rate_&'] = df['funding_rate'] * 100



In [19]:
PRICE = "perp_close"
FUND  = "funding_rate"
CAP   = 0.0001    # ~0.01%
HIGH_N = 12
LOW_N  = 18       # ~3 days on 4H bars
TAKER  = 0.0004   # taker fee rate per side (0.04%)
NOTIONAL = 10_000  # fixed USD notional per trade
INCLUDE_FUNDING_CF = True

# ====== PREP & SIGNALS (build if not already) ======
bt = df.copy()
if not pd.api.types.is_datetime64_any_dtype(bt.index):
    bt.index = pd.to_datetime(bt.index, utc=True)
bt = bt.sort_index()

bt["roll_high"] = bt[PRICE].rolling(HIGH_N, min_periods=HIGH_N).max()
bt["fund_lowN"] = bt[FUND].rolling(LOW_N,  min_periods=LOW_N).min()

bt["sig_top"]    = (bt[FUND] >= CAP - 1e-12) & (bt[PRICE] >= bt["roll_high"])
bt["sig_bottom"] = (bt[FUND] <= bt["fund_lowN"])

bt["sig_top_edge"]    = bt["sig_top"] & (~bt["sig_top"].shift(1).fillna(False))
bt["sig_bottom_edge"] = bt["sig_bottom"] & (~bt["sig_bottom"].shift(1).fillna(False))

# ====== BACKTEST (flip on opposite signal, next-bar execution) ======
trades = []
pos = 0              # +1 long, -1 short, 0 flat
qty = 0.0            # contracts, sized by notional at entry
entry_px = np.nan
entry_time = None
fund_acc_usd = 0.0   # funding cashflow in USD

idx = bt.index
N = len(bt)

def px_at(i):  # you can switch to 'open' if you have it
    return float(bt.iloc[i][PRICE])

for i in range(N - 1):  # need i+1 to execute
    want_long  = bool(bt.iloc[i]["sig_bottom_edge"])
    want_short = bool(bt.iloc[i]["sig_top_edge"])

    if pos == 0:
        if want_long ^ want_short:
            side = +1 if want_long else -1
            px = px_at(i+1)
            pos = side
            entry_px = px
            entry_time = idx[i+1]
            qty = (NOTIONAL / px) * pos                 # quantity sized to $1k notional
            fund_acc_usd = 0.0
            # entry fee (USD)
            entry_fee_usd = TAKER * NOTIONAL
            # we’ll attach fees to the trade on exit
            pending_entry_fee = entry_fee_usd
        continue

    # accumulate funding cashflow (approx: rate * notional * sign)
    if INCLUDE_FUNDING_CF and pd.notna(bt.iloc[i][FUND]):
        fund_acc_usd += float(bt.iloc[i][FUND]) * NOTIONAL * pos

    flip_to_long  = (pos == -1) and want_long
    flip_to_short = (pos == +1) and want_short
    if flip_to_long or flip_to_short:
        # exit current at next bar
        exit_px = px_at(i+1)
        exit_time = idx[i+1]

        # price PnL in USD = qty * (exit - entry)
        pnl_price_usd = qty * (exit_px - entry_px)
        pnl_funding_usd = fund_acc_usd
        exit_fee_usd = TAKER * NOTIONAL
        pnl_usd = pnl_price_usd + pnl_funding_usd - (pending_entry_fee + exit_fee_usd)

        trades.append({
            "entry_time": entry_time,
            "side": "LONG" if pos == 1 else "SHORT",
            "entry_price": entry_px,
            "exit_time": exit_time,
            "exit_price": exit_px,
            "bars_held": (bt.index.get_loc(exit_time) - bt.index.get_loc(entry_time)),
            "pnl_price_usd": pnl_price_usd,
            "funding_usd": pnl_funding_usd,
            "fees_usd": -(pending_entry_fee + exit_fee_usd),
            "pnl_total_usd": pnl_usd,
        })

        # flip: open new opposite at the same next bar
        new_side = +1 if flip_to_long else -1
        px_new = exit_px
        pos = new_side
        entry_px = px_new
        entry_time = exit_time
        qty = (NOTIONAL / px_new) * pos
        fund_acc_usd = 0.0
        pending_entry_fee = TAKER * NOTIONAL  # entry fee for new trade

# Close any open trade at last bar
if pos != 0:
    exit_px = px_at(N-1)
    exit_time = idx[N-1]
    pnl_price_usd = qty * (exit_px - entry_px)
    pnl_funding_usd = fund_acc_usd
    exit_fee_usd = TAKER * NOTIONAL
    pnl_usd = pnl_price_usd + pnl_funding_usd - (pending_entry_fee + exit_fee_usd)
    trades.append({
        "entry_time": entry_time,
        "side": "LONG" if pos == 1 else "SHORT",
        "entry_price": entry_px,
        "exit_time": exit_time,
        "exit_price": exit_px,
        "bars_held": (bt.index.get_loc(exit_time) - bt.index.get_loc(entry_time)),
        "pnl_price_usd": pnl_price_usd,
        "funding_usd": pnl_funding_usd,
        "fees_usd": -(pending_entry_fee + exit_fee_usd),
        "pnl_total_usd": pnl_usd,
    })

log = pd.DataFrame(trades).sort_values("entry_time")
log["cum_pnl_usd"] = log["pnl_total_usd"].cumsum()

summary = {
    "trades": int(len(log)),
    "win_rate": float((log["pnl_total_usd"] > 0).mean()) if len(log) else np.nan,
    "avg_trade_usd": float(log["pnl_total_usd"].mean()) if len(log) else np.nan,
    "median_trade_usd": float(log["pnl_total_usd"].median()) if len(log) else np.nan,
    "total_pnl_usd": float(log["pnl_total_usd"].sum()) if len(log) else 0.0,
    "max_dd_usd": float((log["cum_pnl_usd"].cummax() - log["cum_pnl_usd"]).max()) if len(log) else 0.0,
}

print(summary)
print(log.head(12))

{'trades': 20, 'win_rate': 0.6, 'avg_trade_usd': 100.35540867348496, 'median_trade_usd': 125.0463386316757, 'total_pnl_usd': 2007.108173469699, 'max_dd_usd': 358.46062422869846}
                  entry_time   side  entry_price                 exit_time  \
0  2025-05-09 04:00:00+00:00  SHORT     102419.5 2025-05-11 12:00:00+00:00   
1  2025-05-11 12:00:00+00:00   LONG     104613.7 2025-05-18 20:00:00+00:00   
2  2025-05-18 20:00:00+00:00  SHORT     103834.0 2025-05-21 00:00:00+00:00   
3  2025-05-21 00:00:00+00:00   LONG     106800.1 2025-05-22 04:00:00+00:00   
4  2025-05-22 04:00:00+00:00  SHORT     111755.9 2025-05-24 04:00:00+00:00   
5  2025-05-24 04:00:00+00:00   LONG     108306.8 2025-05-26 12:00:00+00:00   
6  2025-05-26 12:00:00+00:00  SHORT     109736.7 2025-05-27 20:00:00+00:00   
7  2025-05-27 20:00:00+00:00   LONG     109822.8 2025-07-11 04:00:00+00:00   
8  2025-07-11 04:00:00+00:00  SHORT     116839.5 2025-07-14 00:00:00+00:00   
9  2025-07-14 00:00:00+00:00   LONG     11

/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_83806/1206497129.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bt["sig_top_edge"]    = bt["sig_top"] & (~bt["sig_top"].shift(1).fillna(False))
/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_83806/1206497129.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bt["sig_bottom_edge"] = bt["sig_bottom"] & (~bt["sig_bottom"].shift(1).fillna(False))
